In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf /content/drive/Shareddrives/DA231-2022-public/spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install yfinance
!pip install plotly
!pip install --upgrade pandas-datareader
!pip install kafka-python


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 246 kB 5.0 MB/s 


In [ ]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder\
         .master("local")\
         .appName("Colab")\
         .config('spark.ui.port', '4050')\
         .getOrCreate()
# Raw Package
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import col
#Data Source
import yfinance as yf
#Data viz
import plotly.graph_objs as go
import pandas_datareader as web
import pyspark.sql.functions as F
import time
from pyspark.sql.window import Window


In [ ]:

from json import dumps,loads
from kafka import KafkaProducer
import pandas as pd
import pandas_datareader as web
from time import sleep

producer = KafkaProducer(bootstrap_servers=['192.168.1.15:9092'],api_version=(0,11,5),value_serializer=lambda x:dumps(x).encode('utf-8'))
df = yf.download(tickers='AAPL', period='2d', interval='1m')
date= df.index
df['Date']=date
print(df.sample(1).to_dict(orient='records')[0])
while True:
  dict_stock = df.sample(1).to_dict(orient='records')[0]
  producer.send('RTSTCKDATA',value=dict_stock)
  sleep(1)

producer.flush()

producer.send('demo_test', value ={'msg': 'all is well'})

[*********************100%***********************]  1 of 1 completed
{'Open': 142.77000427246094, 'High': 143.0800018310547, 'Low': 142.75120544433594, 'Close': 142.9600067138672, 'Adj Close': 142.9600067138672, 'Volume': 354198, 'Date': Timestamp('2022-12-09 10:31:00')}


ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=192.168.1.15:9092 <connecting> [IPv4 ('192.168.1.15', 9092)]> timed out
ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=192.168.1.15:9092 <connecting> [IPv4 ('192.168.1.15', 9092)]> timed out
ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=192.168.1.15:9092 <connecting> [IPv4 ('192.168.1.15', 9092)]> timed out
ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=192.168.1.15:9092 <connecting> [IPv4 ('192.168.1.15', 9092)]> timed out
ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=192.168.1.15:9092 <connecting> [IPv4 ('192.168.1.15', 9092)]> timed out
ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0 host=192.168.1.15:9092 <connecting> [IPv4 ('192.168.1.15', 9092)]> timed out
ERROR:kafka.conn:Connection attempt to <BrokerConnection node_id=bootstrap-0

KafkaTimeoutError: ignored

In [ ]:
#Interval required 5 minutes
data = yf.download(tickers='AAPL', period='2d', interval='1m')
date = data.index
data['Date']=date
sdata = spark.createDataFrame(data) 
sdata.printSchema()


[*********************100%***********************]  1 of 1 completed
root
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Date: timestamp (nullable = true)



In [ ]:
def movingAverage(inDF, colName, window):
  outDF = None  
  inDFI = inDF.withColumn("Id",F.monotonically_increasing_id())
   #null_condition = F.col(colName).isNull()
  #inDFP = inDFT.withColumn("percent_rank",F.when(null_condition, F.lit(None)).otherwise( F.percent_rank().over(Window.partitionBy(inDFT['City'],F.when(null_condition, 1).otherwise(0)).orderBy(inDFT[colName]))))
  windowspec = Window.partitionBy().orderBy("Date").rowsBetween(Window.currentRow-window,Window.currentRow)
  outDF=inDFI.withColumn('MovingAvg'+colName, F.format_number((F.mean(colName).over(windowspec)),3))
  outDF.show()
  return outDF
  
def sumdata(inDF, colName, window):
  outDF = None  
  inDFI = inDF.withColumn("Id",F.monotonically_increasing_id())
   #null_condition = F.col(colName).isNull()
  #inDFP = inDFT.withColumn("percent_rank",F.when(null_condition, F.lit(None)).otherwise( F.percent_rank().over(Window.partitionBy(inDFT['City'],F.when(null_condition, 1).otherwise(0)).orderBy(inDFT[colName]))))
  windowspec = Window.partitionBy().orderBy("Date").rowsBetween(Window.currentRow-window,Window.currentRow)
  outDF=inDFI.withColumn('Sum'+colName, F.format_number((F.sum(colName).over(windowspec)),3))
  outDF.show()
  return outDF 

In [ ]:
DF = movingAverage(sdata,'Close',60)
DF1 = sumdata(sdata,'Volume',30)
print(DF1)

+------------------+------------------+------------------+------------------+------------------+-------+-------------------+---+--------------+
|              Open|              High|               Low|             Close|         Adj Close| Volume|               Date| Id|MovingAvgClose|
+------------------+------------------+------------------+------------------+------------------+-------+-------------------+---+--------------+
|142.36000061035156|142.66749572753906|142.11000061035156| 142.5050048828125| 142.5050048828125|1838425|2022-12-08 09:30:00|  0|       142.505|
|142.50289916992188|142.55999755859375|142.02000427246094|142.38999938964844|142.38999938964844| 286139|2022-12-08 09:31:00|  1|       142.448|
|142.37159729003906| 142.4199981689453|             142.0|142.02000427246094|142.02000427246094| 186244|2022-12-08 09:32:00|  2|       142.305|
|142.00999450683594|142.02000427246094|141.80999755859375| 141.9282989501953| 141.9282989501953| 216083|2022-12-08 09:33:00|  3|       1

In [ ]:
inputDF= spark.readStream.format("kafka").option("kafka.bootstrap.servers","192.168.1.15:9092").option('subscribe','RTSTCKDATA').load()
rows = inputDF.select('Volume','Date')
StreamingQueryDF = (rows.groupBy('Volume',Window('Date','15 minute').sum()).writeStream.format("console").outputMode('complete').trigger(processingTime='1 second').start())
StreamingQueryDF.awaitTermination()


AnalysisException: ignored